In [1]:
import pandas as pd
import numpy as np

In [2]:
y = pd.read_csv("cls_labelWithfea.csv")
print(len(y.loc[y["label"] == 0]))
print(len(y.loc[y["label"] == 1]))
y = y["label"]

32827
2418


In [3]:
fea_concat = pd.read_csv("sym_mesh_concat.csv",index_col=0)
print(fea_concat.shape)
fea_concat.head()

(35245, 692)


,0,1,2,3,4,5,6,7,8,9,...,38.1.1,39.1.1,40.1.1,41.1.1,42.1.1,43.1.1,44.1.1,45.1.1,46.1.1,47.1.1
0,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve

from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

from sklearn.utils import shuffle

from imblearn.under_sampling import RandomUnderSampler # Random UnderSampler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve,auc

## Negative sampling + split test set

In [50]:
import random as python_random
seed = 1
np.random.seed(seed) # This is necessary for starting Numpy generated random numbers in a well-defined initial state.
python_random.seed(seed)# This is necessary for starting core Python generated random numbers in a well-defined state.

rus = RandomUnderSampler(sampling_strategy=1,random_state=seed)
rus_x, rus_y = rus.fit_resample(fea_concat ,y) #rus_y(Series)
print('负采样后数据集大小:',len(rus_y))

fea_train,fea_test,y_train,y_test=train_test_split(
                rus_x,
                rus_y,
                test_size=0.2, shuffle=True)

print(fea_train.shape,fea_test.shape)

负采样后数据集大小: 4836
(3868, 692) (968, 692)


## LightGBM

### 5-fold

In [39]:
#LightGBM
import lightgbm as lgb

params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': {'auc'},
        'max_depth':10,
        'num_leaves':31,
        'learning_rate':0.05,
        #'feature_fraction':0.9,
        'bagging_fraction':0.8,
        'bagging_freq':5,
        'verbose':-1,
        'is_provide_training_metric':False,
        
        #'scale_pos_weight':2
        #'is_unbalance':True,
        'min_data_in_leaf':20
}


kf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=seed)

test_auc_fold = []
test_aupr_fold = []
test_acc_fold = []

best_iterations=[]

for train_index, test_index in kf.split(fea_train, y_train):
    X_train, Y_train = fea_train.iloc[train_index], y_train.iloc[train_index]
    X_test, Y_test = fea_train.iloc[test_index], y_train.iloc[test_index]
    train_data = lgb.Dataset(X_train, Y_train)
    eval_data = lgb.Dataset(X_test, Y_test)
    
    evals_result={}
    #num_boost_round=5000
    gbm = lgb.train(params, train_data, early_stopping_rounds=50,
                    num_boost_round=10000, valid_sets=eval_data ,valid_names=["train","test"],evals_result=evals_result)
    #early_stopping_rounds=200
    Y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    best_iterations.append(gbm.best_iteration)
    
    test_auc = roc_auc_score(Y_test, Y_pred)#AUROC
    test_aupr = average_precision_score(Y_test, Y_pred)#AUPR
    
    Y_pred = [1 if i>=0.5 else 0 for i in Y_pred]# trans binary
    test_acc = accuracy_score(Y_test, Y_pred) #ACC

    test_auc_fold.append(test_auc)
    test_aupr_fold.append(test_aupr)    
    test_acc_fold.append(test_acc)
    
test_auc_fold.append(np.mean(test_auc_fold))
test_aupr_fold.append(np.mean(test_aupr_fold))
test_acc_fold.append(np.mean(test_acc_fold))

print('avg_best_iteration: ',np.mean(best_iterations))
print('AUROC: ',test_auc_fold[:-1])
print('-----AverageAUC: ',test_auc_fold[-1])
print('AUPR: ',test_aupr_fold[:-1])
print('-----AverageAUPR: ',test_aupr_fold[-1])
print('ACC: ',test_acc_fold[:-1])
print('-----AverageACC: ',test_acc_fold[-1])

[1]	train's auc: 0.722479
Training until validation scores don't improve for 50 rounds
[2]	train's auc: 0.740532
[3]	train's auc: 0.745052
[4]	train's auc: 0.75203
[5]	train's auc: 0.748561
[6]	train's auc: 0.754764
[7]	train's auc: 0.75811
[8]	train's auc: 0.765391
[9]	train's auc: 0.765829
[10]	train's auc: 0.768102
[11]	train's auc: 0.770333
[12]	train's auc: 0.769247
[13]	train's auc: 0.776182
[14]	train's auc: 0.780015
[15]	train's auc: 0.781377
[16]	train's auc: 0.784609
[17]	train's auc: 0.786792
[18]	train's auc: 0.786819
[19]	train's auc: 0.787717
[20]	train's auc: 0.786348
[21]	train's auc: 0.786762
[22]	train's auc: 0.786889
[23]	train's auc: 0.786722
[24]	train's auc: 0.787136
[25]	train's auc: 0.787567
[26]	train's auc: 0.790131
[27]	train's auc: 0.790204
[28]	train's auc: 0.791393
[29]	train's auc: 0.792274
[30]	train's auc: 0.791373
[31]	train's auc: 0.792348
[32]	train's auc: 0.793269
[33]	train's auc: 0.793583
[34]	train's auc: 0.795005
[35]	train's auc: 0.796401
[36]	

[299]	train's auc: 0.835597
[300]	train's auc: 0.83545
[301]	train's auc: 0.835557
[302]	train's auc: 0.836004
[303]	train's auc: 0.836251
[304]	train's auc: 0.836064
[305]	train's auc: 0.836138
[306]	train's auc: 0.836472
[307]	train's auc: 0.836599
[308]	train's auc: 0.836579
[309]	train's auc: 0.836866
[310]	train's auc: 0.83734
[311]	train's auc: 0.837413
[312]	train's auc: 0.837507
[313]	train's auc: 0.837587
[314]	train's auc: 0.837527
[315]	train's auc: 0.837447
[316]	train's auc: 0.837587
[317]	train's auc: 0.837473
[318]	train's auc: 0.837821
[319]	train's auc: 0.837861
[320]	train's auc: 0.837967
[321]	train's auc: 0.838435
[322]	train's auc: 0.838522
[323]	train's auc: 0.838435
[324]	train's auc: 0.838689
[325]	train's auc: 0.838789
[326]	train's auc: 0.838669
[327]	train's auc: 0.838482
[328]	train's auc: 0.838341
[329]	train's auc: 0.838428
[330]	train's auc: 0.838214
[331]	train's auc: 0.837947
[332]	train's auc: 0.83774
[333]	train's auc: 0.837246
[334]	train's auc: 0.83

[241]	train's auc: 0.823037
[242]	train's auc: 0.823324
[243]	train's auc: 0.823898
[244]	train's auc: 0.824232
[245]	train's auc: 0.824679
[246]	train's auc: 0.824927
[247]	train's auc: 0.824927
[248]	train's auc: 0.824927
[249]	train's auc: 0.825501
[250]	train's auc: 0.825327
[251]	train's auc: 0.825674
[252]	train's auc: 0.825975
[253]	train's auc: 0.826028
[254]	train's auc: 0.825962
[255]	train's auc: 0.826028
[256]	train's auc: 0.826302
[257]	train's auc: 0.826896
[258]	train's auc: 0.827284
[259]	train's auc: 0.827738
[260]	train's auc: 0.828205
[261]	train's auc: 0.828165
[262]	train's auc: 0.828172
[263]	train's auc: 0.827958
[264]	train's auc: 0.827838
[265]	train's auc: 0.827551
[266]	train's auc: 0.827711
[267]	train's auc: 0.827771
[268]	train's auc: 0.827658
[269]	train's auc: 0.827544
[270]	train's auc: 0.827931
[271]	train's auc: 0.827744
[272]	train's auc: 0.827618
[273]	train's auc: 0.827691
[274]	train's auc: 0.827424
[275]	train's auc: 0.82697
[276]	train's auc: 0.

[154]	train's auc: 0.808861
[155]	train's auc: 0.808567
[156]	train's auc: 0.808727
[157]	train's auc: 0.808841
[158]	train's auc: 0.808714
[159]	train's auc: 0.808353
[160]	train's auc: 0.808093
[161]	train's auc: 0.807459
[162]	train's auc: 0.807425
[163]	train's auc: 0.807158
[164]	train's auc: 0.807419
[165]	train's auc: 0.807232
[166]	train's auc: 0.807666
[167]	train's auc: 0.807993
[168]	train's auc: 0.808213
[169]	train's auc: 0.807973
[170]	train's auc: 0.807939
[171]	train's auc: 0.807946
[172]	train's auc: 0.807726
[173]	train's auc: 0.807545
[174]	train's auc: 0.807679
[175]	train's auc: 0.807893
[176]	train's auc: 0.808006
[177]	train's auc: 0.808146
[178]	train's auc: 0.808433
[179]	train's auc: 0.809008
[180]	train's auc: 0.808928
[181]	train's auc: 0.809108
[182]	train's auc: 0.809034
[183]	train's auc: 0.809181
[184]	train's auc: 0.808634
[185]	train's auc: 0.808921
[186]	train's auc: 0.809335
[187]	train's auc: 0.809615
[188]	train's auc: 0.809876
[189]	train's auc: 0

[38]	train's auc: 0.788734
[39]	train's auc: 0.788708
[40]	train's auc: 0.789685
[41]	train's auc: 0.790053
[42]	train's auc: 0.791827
[43]	train's auc: 0.79243
[44]	train's auc: 0.794378
[45]	train's auc: 0.79478
[46]	train's auc: 0.796165
[47]	train's auc: 0.795456
[48]	train's auc: 0.796842
[49]	train's auc: 0.79891
[50]	train's auc: 0.798863
[51]	train's auc: 0.800617
[52]	train's auc: 0.801882
[53]	train's auc: 0.802585
[54]	train's auc: 0.803067
[55]	train's auc: 0.803101
[56]	train's auc: 0.804212
[57]	train's auc: 0.805591
[58]	train's auc: 0.807037
[59]	train's auc: 0.808015
[60]	train's auc: 0.808202
[61]	train's auc: 0.809313
[62]	train's auc: 0.810264
[63]	train's auc: 0.811221
[64]	train's auc: 0.812192
[65]	train's auc: 0.812915
[66]	train's auc: 0.813156
[67]	train's auc: 0.813638
[68]	train's auc: 0.813933
[69]	train's auc: 0.814455
[70]	train's auc: 0.814649
[71]	train's auc: 0.815841
[72]	train's auc: 0.816805
[73]	train's auc: 0.81726
[74]	train's auc: 0.817548
[75]	

[1]	train's auc: 0.727493
Training until validation scores don't improve for 50 rounds
[2]	train's auc: 0.734964
[3]	train's auc: 0.740136
[4]	train's auc: 0.743801
[5]	train's auc: 0.743366
[6]	train's auc: 0.755727
[7]	train's auc: 0.760259
[8]	train's auc: 0.768306
[9]	train's auc: 0.770217
[10]	train's auc: 0.77172
[11]	train's auc: 0.772202
[12]	train's auc: 0.771248
[13]	train's auc: 0.771617
[14]	train's auc: 0.773605
[15]	train's auc: 0.776909
[16]	train's auc: 0.77893
[17]	train's auc: 0.778873
[18]	train's auc: 0.780842
[19]	train's auc: 0.781873
[20]	train's auc: 0.780189
[21]	train's auc: 0.78216
[22]	train's auc: 0.783135
[23]	train's auc: 0.783921
[24]	train's auc: 0.784885
[25]	train's auc: 0.78531
[26]	train's auc: 0.785146
[27]	train's auc: 0.785337
[28]	train's auc: 0.785719
[29]	train's auc: 0.78448
[30]	train's auc: 0.787452
[31]	train's auc: 0.788028
[32]	train's auc: 0.788001
[33]	train's auc: 0.78765
[34]	train's auc: 0.788032
[35]	train's auc: 0.789521
[36]	trai

- avg_best_iteration:  319.6
- AUROC:
[0.8387887286324787, 0.8326655982905984, 0.8196647970085471, 0.8501030955441302, 0.838709029562982]
-----AverageAUC:
0.8359862498077473
- AUPR:
[0.8302394936880727, 0.8312328552347071, 0.8150530701675709, 0.8558679698236631, 0.8367287565878734]
-----AverageAUPR
0.8338244291003774
- ACC:
[0.7777777777777778, 0.7674418604651163, 0.7325581395348837, 0.7697283311772316, 0.7723156532988357]
-----AverageACC
0.763964352450769

### Test

In [51]:
#y_test-binary-int;y_pred-proba-float
def test_metric(y_test,y_pred):
    y_test=np.array(y_test)
    y_pred=np.array(y_pred)
    
    y_pred_bina = np.around(y_pred,0).astype(int)
    class_acc=accuracy_score(y_test,y_pred_bina)
    
    y_test_int = y_test.astype(int)
    fpr,tpr,threshold=roc_curve(y_test_int,y_pred)#y_test-int(ndarray)
    print(len(fpr))
    roc_auc=auc(fpr,tpr)
    
    print('accuracy {}\t'.format(str(class_acc)), 'AUC:{}\n'.format(roc_auc))

    return np.array(class_acc),np.array(roc_auc)

In [52]:
#Using optimal number of iterations using the 5-fold of cross-validation

train_data = lgb.Dataset(fea_train, y_train)

gbm_test = lgb.train(params, 
                train_data, 
                num_boost_round=np.mean(best_iterations).astype(int)
                )

predictions = gbm_test.predict(fea_test, num_iteration=np.mean(best_iterations).astype(int))

accuracy,AUC = test_metric(y_test,predictions)

print('predictions lenth：{}\n'.format(len(predictions)),
    'accuracy {} \n'.format(accuracy),
     'AUC {} \n'.format(AUC))


352
accuracy 0.7479338842975206	 AUC:0.8202683072716398

predictions lenth：968
 accuracy 0.7479338842975206 
 AUC 0.8202683072716398 



In [53]:
AUC_data = np.array([list(y_test),list(predictions)])
print(AUC_data.shape)
AUC_data_df=pd.DataFrame(AUC_data.T,columns=["label","predict"])
AUC_data_df.head()

(2, 968)


,label,predict
0,0.0,0.285716
1,0.0,0.785368
2,0.0,0.094459
3,1.0,0.821075
4,0.0,0.177990


In [54]:
# AUC_data_df.to_csv("auc_data_lightgbm.csv")